### Update information: Input required
#### Note: quotation marks should be included. Change values inside of the quotes.

In [1]:
AY = 'AY 2022-2023'
CO = 'Brain and Behavior'

#### Required files: input.xlsx, faculty.xlsx
### Faculty (Didactic) Data Automation

In [2]:
# initiate
import numpy as np
import pandas as pd
from datetime import datetime
import datetime
import time
import warnings
warnings.simplefilter("ignore")

# start timer
tic = time.time()

# import dataset
read = pd.read_excel('input.xlsx', engine='openpyxl')
fac_df = pd.read_excel('faculty.xlsx', engine='openpyxl')

#filter in applicable data only: "0/IP Address" rows only
readfile = read[(read.Status.isin(['Response Type', 0]))]

# subset survey questions
q_subset = readfile.loc[:,readfile.columns.str.contains('.*Q2.11')]

# create a list of columns
q_cols_subset = list(q_subset.columns)

In [3]:
df1 = pd.DataFrame([])
df2 = pd.DataFrame([])
df1['Faculty_FirstName'] =''
df1['Faculty_LastName']= ''


for k in q_cols_subset:
      
    q_text_c = pd.DataFrame(q_subset[k][1:,].dropna())
    q_text_c.columns = ['Comments']
    
    df_fac1 = pd.concat([df1, q_text_c], axis=1)
    
    q_text_f = str(q_subset[k][0])
        
    sub1 = "Dr."
    sub2 = "</"

    # getting index of substrings
    idx1 = q_text_f.index(sub1)
    idx2 = q_text_f.index(sub2)

    res = ''
    # getting elements in between
    for idx in range(idx1 + len(sub1) + 1, idx2):
        res = res + q_text_f[idx]

    # result
    Faculty_FullName = res.split()
    df_fac1['Faculty_FirstName'] =str(Faculty_FullName[0])
    df_fac1['Faculty_LastName']= str(Faculty_FullName[-1])
    
    df2 = pd.concat([df2, df_fac1], axis=0)

# merge faculty db with the faculty name column
df2['Faculty'] = df2.Faculty_LastName +', ' + df2.Faculty_FirstName
df3 = pd.merge(df2, fac_df, left_on='Faculty', right_on='Name', how='left').drop('Name', axis=1)

# populate columns for AY, Class, Block
df3['AcademicYear'] = AY
df3['Course'] = CO
df3 = df3[['NetID', 'GWID', 'Department', 'AcademicYear', 'Course',
           'Faculty', 'Faculty_FirstName', 'Faculty_LastName', 'Comments']]

# export
df3.to_excel('output_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + '.xlsx', index = False)

# end timer             
toc = time.time() - tic
print('All tasks completed in %.2f s.' % (toc))

All tasks completed in 1.63 s.
